In [1]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json
from scripts.result_export import generate_html
from scripts.price_process import price_process


# Read Data

In [2]:
excel_mode = 1

In [3]:
if excel_mode == 1:
    # Read price data from Excel
    energy_price = pd.read_excel('data/btm/energy_price.xlsx', index_col=0)
    demand_price = pd.read_excel('data/btm/demand_price.xlsx', index_col=0)
    load_profile = pd.read_excel('data/btm/load_profile.xlsx', index_col=0)

    model_time_period = 24    # periods
    dt = 1

    price_df = energy_price.copy()

In [4]:
# Parameters
schedule_data = pd.read_excel('data/schedule_data_s1.xlsx', sheet_name= None, index_col=0)

λ_min = schedule_data['soc_limit']['min']
λ_max = schedule_data['soc_limit']['max']
theta = schedule_data['availability']

z = {}
s = {}
z['arb'], z['reg'], z['pres'], z['cres'], z['opp'] = schedule_data['basis'].value.values
s['reg'] = schedule_data['basis'].loc['regulation', 'sym']


In [5]:
filename = "test"

z['opp'] = 0
# s['reg'] = 1

cap_power = 1  # Example value
cap_energy = 4  # Example value

Seff = 0.9995
Ceff = 0.9
Deff = 0.9


λ_0 = 0.5
initial_soc = λ_0 * cap_energy

kappa = 0.95     # regulation performance score
beta = 0.15       # activated portion for reserve


cycle_life = 3650
calendar_life = 10
total_time_period = len(price_df)
num_slices = int(total_time_period // model_time_period)     # number of time slices to model

η = cycle_life/calendar_life/365                 # number of cycle per time slice

# Model

In [6]:
def optimize_revenue(initial_soc, periodic_data, current_period, first_period, last_period):

    energy_price, load_profile = periodic_data
    total_time_period = len(energy_price)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_elec = {t: energy_price['Price'].values[t-1] for t in T}
    L_elec = {t: load_profile['Load'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Define the model
    model = ConcreteModel()

    # Define variables
    model.P_grid = Var(T, within=NonNegativeReals)
    model.P_grid_L = Var(T, within=NonNegativeReals)
    model.P_grid_S = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.P_S_L = Var(T, within=NonNegativeReals, bounds = (0, cap_power))
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))

    # Objective function
    def total_cost(model):
        return sum(model.P_grid[t] * p_elec[t] for t in T)
    model.total_cost = Objective(rule=total_cost, sense=minimize)

    # Constraints

    # Constraint 1: Total grid power
    def grid_power_rule(model, t):
        return model.P_grid[t] == model.P_grid_L[t] + model.P_grid_S[t]
    model.grid_power_constraint = Constraint(T, rule=grid_power_rule)

    # Constraint 2: Load balance
    def load_balance_rule(model, t):
        return model.P_grid_L[t] + model.P_S_L[t] == L_elec[t]
    model.load_balance_constraint = Constraint(T, rule=load_balance_rule)

    # Constraint 3: Storage energy balance
    def storage_energy_balance_rule(model, t):
        if t == 0:
                return model.SOC[t] == initial_soc
        else:
            return model.SOC[t] == model.SOC[t-1] * Seff + (model.P_grid_S[t] * Ceff - model.P_S_L[t] / Deff) * dt
    model.storage_energy_balance_constraint = Constraint(SOC_T, rule=storage_energy_balance_rule)

    # Solver
    solver = SolverFactory('glpk')

    # Solve the model
    results = solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    grid_power = [model.P_grid[t].value for t in T]  # Charging as negative
    battery_charge = [model.P_grid_S[t].value for t in T]  # Charging as negative
    batter_discharge = [model.P_S_L[t].value for t in T]  # Discharging as positive
    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    return model.total_cost(), grid_power, battery_charge, batter_discharge, soc_schedule 


# Run

In [7]:
# Lists to store results
total_revenue = 0

all_grid_schedules = []
all_charging_schedules = []
all_discharging_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices - 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_energy_price = energy_price[p*model_time_period:(p+1)*model_time_period]    
    periodic_load_profile = load_profile[p*model_time_period:(p+1)*model_time_period]
    obj, grid_schedule, charging_schedule, discharging_schedule, soc_schedule = optimize_revenue(initial_soc, [periodic_energy_price, periodic_load_profile], p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += obj

    all_grid_schedules.extend(grid_schedule)
    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

watch 0
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\angelmah\AppData\Local\Temp\tmp_9jsmmrg.glpk.raw --wglp
 C:\Users\angelmah\AppData\Local\Temp\tmp2p0fn5pj.glpk.glp --cpxlp C:\Users\angelmah\AppData\Local\Temp\tmp3fmm4nbu.pyomo.lp
Reading problem data from 'C:\Users\angelmah\AppData\Local\Temp\tmp3fmm4nbu.pyomo.lp'...
73 rows, 121 columns, 217 non-zeros
590 lines were read
Writing problem data to 'C:\Users\angelmah\AppData\Local\Temp\tmp2p0fn5pj.glpk.glp'...
536 lines were written
GLPK Simplex Optimizer, v4.65
73 rows, 121 columns, 217 non-zeros
Preprocessing...
24 rows, 48 columns, 71 non-zeros
Scaling...
 A: min|aij| =  9.000e-01  max|aij| =  1.111e+00  ratio =  1.235e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 24
      0: obj =   2.796803911e+03 inf =   1.999e+00 (1)
      9: obj =   2.877160368e+03 inf =   0.000e+00 (0)
*    36: obj =   2.601120975e+03 inf =   0.000e+00 (0)
OP

In [8]:
# Create a DataFrame for the results
data = {
    'time': price_df.index[:len(all_grid_schedules)],
    'grid': all_grid_schedules,
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

#result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

,time,grid,charge,discharge,soc,net_discharge,soc_percent
0,2018-01-01 00:00:00,0.904036,0.0,0.000000,2.000000,0.000000,0.500000
1,2018-01-01 01:00:00,0.290359,0.0,0.000000,1.999000,0.000000,0.499750
2,2018-01-01 02:00:00,0.290359,0.0,0.000000,1.998001,0.000000,0.499500
3,2018-01-01 03:00:00,0.291143,0.0,0.000000,1.997001,0.000000,0.499250
4,2018-01-01 04:00:00,0.291143,0.0,0.000000,1.996003,0.000000,0.499001
...,...,...,...,...,...,...,...
8755,12/31/18 20:00,0.000000,0.0,0.291143,0.323655,0.291143,0.080914
8756,12/31/18 21:00,0.291143,-0.0,0.000000,0.000000,0.000000,0.000000
8757,12/31/18 22:00,0.291143,-0.0,0.000000,0.000000,0.000000,0.000000
8758,12/31/18 23:00,0.291143,-0.0,0.000000,0.000000,0.000000,0.000000


In [9]:
result_df.head(20)

,time,grid,charge,discharge,soc,net_discharge,soc_percent
0,2018-01-01 00:00:00,9.040359e-01,0.000000,0.000000,2.000000,0.000000,0.500000
1,2018-01-01 01:00:00,2.903587e-01,0.000000,0.000000,1.999000,0.000000,0.499750
2,2018-01-01 02:00:00,2.903587e-01,0.000000,0.000000,1.998001,0.000000,0.499500
3,2018-01-01 03:00:00,2.911435e-01,0.000000,0.000000,1.997001,0.000000,0.499250
4,2018-01-01 04:00:00,2.911435e-01,0.000000,0.000000,1.996003,0.000000,0.499001
5,2018-01-01 05:00:00,2.911435e-01,0.000000,0.000000,1.995005,0.000000,0.498751
6,2018-01-01 06:00:00,2.911435e-01,0.000000,0.000000,1.994007,0.000000,0.498502
7,2018-01-01 07:00:00,2.911435e-01,0.000000,0.000000,1.993010,0.000000,0.498253
8,2018-01-01 08:00:00,5.255078e-01,0.235149,0.000000,1.992014,-0.235149,0.498003
9,2018-01-01 09:00:00,1.290359e+00,1.000000,0.000000,2.202652,-1.000000,0.550663


In [10]:
p=1
load_profile[p*model_time_period:(p+1)*model_time_period]

,Load
Time,
2018-02-01 00:00:00,0.769058
2018-02-01 01:00:00,0.768274
2018-02-01 02:00:00,0.769058
2018-02-01 03:00:00,0.779260
2018-02-01 04:00:00,0.780830
2018-02-01 05:00:00,0.766704
2018-02-01 06:00:00,0.737668
2018-02-01 07:00:00,0.713341
2018-02-01 08:00:00,0.711771


In [11]:
price_df.index.names = ['time']
price_data_ex = price_df.reset_index().to_json(orient='records', date_format='iso')
result_data_ex = result_df.to_json(orient='records', date_format='iso')

In [12]:
from scripts.result_export import generate_html

BESS_icost = 207
BESS_ecost = 355
BESS_pcost = 153

annual_revenue = total_revenue/num_slices * 365
discount_rate = 0.05
om_percentage = 0.05

generate_html(filename, BESS_icost, BESS_ecost, BESS_pcost, cap_energy, cap_power, annual_revenue, price_data_ex, result_data_ex, revenue_data, discount_rate, om_percentage, calendar_life)

NameError: name 'revenue_data' is not defined

In [ ]:
initial_cost

In [ ]:
BESS_icost = 207000
BESS_ecost = 355000
BESS_pcost = 153000

initial_cost = 207000 + 355000 * cap_energy + 153000*cap_power
annual_cost = initial_cost * 0.05
annual_revenue = total_revenue/num_slices * 365
discount_rate = 0.05
om_percentage = 0.05

In [ ]:
initial_cost

In [ ]:
annual_cost

# Plot

In [ ]:
df = result_df
df = df.set_index(['time'])

#df['d'] = df['time'].dt.date
#df['t'] = df['time'].dt.time
#df = df.set_index(['d', 't']).drop(columns = ['time'])


In [ ]:
start_time = 0
end_time = 48

# Create price figure
fig_price = go.Figure()
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['arb_energy_price'], mode='lines', name='Energy'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['reg_up_price'], mode='lines', name='Regulation Up'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['reg_down_price'], mode='lines', name='Regulation Down'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['pres_capacity_price'], mode='lines', name='Primary Reserve'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['cres_capacity_price'], mode='lines', name='Contingency Reserve'))

fig_price.update_layout(
    title='Prices',
    xaxis_title='Time',
    yaxis_title='$/MWh',
    legend=dict(
        orientation='h',
        x=0,
        y=1.1
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Create charging and discharging figure
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['net_discharge'], mode='lines', name='Net Power Discharge'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['reg_up'], mode='lines', name='Regulation Up'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['reg_down'], mode='lines', name='Regulation Down'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['pres'], mode='lines', name='Primary Reserve'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['cres'], mode='lines', name='Contingency Reserve'))

fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    legend=dict(
        orientation='h',
        x=0,
        y=1.1
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Create state of charge figure
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index[start_time:end_time], y=result_df['soc_percent']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    legend=dict(
        orientation='h',
        x=0,
        y=1.1
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Generate HTML snippets for each figure
price_html = pio.to_html(fig_price, full_html=False)
charge_discharge_html = pio.to_html(fig_charge_discharge, full_html=False)
soc_html = pio.to_html(fig_soc, full_html=False)

# Combine all HTML snippets into a single HTML document
full_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Figures</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>Price Figure</h1>
    {price_html}
    <h1>Charging and Discharging Schedule</h1>
    {charge_discharge_html}
    <h1>State of Charge</h1>
    {soc_html}
</body>
</html>
"""

# Save to an HTML file
with open("result/day/" + filename + ".html", "w",  encoding="utf-8") as file:
    file.write(full_html)


